# Знакомство с word2vec


Меняем значения по заданию

In [ ]:
task1 = ["vodka", "drink"]
task2 = "vodka drink car gem"
task3 = ["empty vessel makes much noise", "idle brain is the devil workshop"]
indexs = [28, 128]

Скачиваем данные (будет долго)



In [ ]:
! pip install -q -U gensim
! gdown --id 0B7XkCwpI5KDYNlNUTTlSS21pQmM
! pip install -q SciPy==1.5.4
! gunzip GoogleNews-vectors-negative300.bin.gz

! wget -c http://vectors.nlpl.eu/repository/20/0.zip
! unzip 0.zip
! head -3 model.txt

import warnings
import gensim
from gensim.models import KeyedVectors
warnings.filterwarnings('ignore')

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM
To: /content/GoogleNews-vectors-negative300.bin.gz
100% 1.65G/1.65G [00:06<00:00, 258MB/s]
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Installing build dependencies ... error
error: subprocess-exited-with-error

× pip subprocess to install build dependencies did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
--2023-05-02 12:20:48--  http://vectors.nlpl.eu/repository/20/0.zip
Resolving vect

In [ ]:
w = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary=True)
w_british = KeyedVectors.load_word2vec_format("model.bin", binary=True)

Запускаем код

In [ ]:
import numpy as np
import pandas as pd
import scipy.spatial.distance
from scipy.stats import spearmanr

# Задание 1
print("Задание 1:")
print("GN: %.3f" % w.distance(task1[0], task1[1]))
print("BR: %.3f" % w_british.distance(task1[0] + "_NOUN", task1[1] + "_NOUN"))
print()

# Задание 2
print("Задание 2:")
text = task2.split()
text_NOUN = []
for i in text:
  text_NOUN.append(i + "_NOUN")

print("GN:", w.doesnt_match(text))
print("BR:", w_british.doesnt_match(text_NOUN).replace("_NOUN", ""))
print()

# Задание 3
print("Задание 3:")

text_split = task3[0].split()
text2_split = task3[1].split()
wtf = {}
wtf2 = {}
new_wtf = []
new_wtf2 = []

for j in range(300):
  for i in text_split:
    if j not in wtf.keys():
      wtf[j] = []
    wtf[j].append(w[i][j])

for i in wtf:
  new_wtf.append(np.mean(wtf[i]))

for j in range(300):
  for i in text2_split:
    if j not in wtf2.keys():
      wtf2[j] = []
    wtf2[j].append(w[i][j])

for i in wtf2:
  new_wtf2.append(np.mean(wtf2[i]))

print("Косинусное растояние GN: %.3f" % scipy.spatial.distance.cosine(new_wtf, new_wtf2))
print()

# Задание 4
print("Задание 4:")
df = pd.read_csv("https://courses.openedu.ru/assets/courseware/v1/e5d93a68121655b8f04aea15b89c130a/asset-v1:ITMOUniversity+AUTTEXTAN+fall_2021_ITMO+type@asset+block/wordsim_similarity_goldstandard.txt", sep="\t", header=None)
df.columns = ["first", "second", "score"]
gn_dist, br_dist, scores = [], [], []
count = 0
in_c = 0

for row in df.iterrows():
  if indexs[0] > in_c or in_c > indexs[1] - 1:
    in_c += 1
    continue
  in_c += 1
    
  w1, w2, s = row[1]["first"], row[1]["second"], row[1]["score"]

  try:
    w_british[w1.lower() + "_NOUN"]
    w_british[w2.lower() + "_NOUN"]

    gn_dist.append(w.similarity(w1, w2))
    br_dist.append(w_british.similarity(w1.lower() + "_NOUN", w2.lower() + "_NOUN"))
    scores.append(s)
    
  except KeyError as e:
    count += 1
    #print(e, "Skipping this word.")

coef, p = spearmanr(gn_dist, scores)
print("GN: %.3f" % coef)
coef, p = spearmanr(br_dist, scores)
print("BR: %.3f" % coef)
print("Удалено пар:", count)
print()

Задание 1:
GN: 0.428
BR: 0.411

Задание 2:
GN: gem
BR: gem

Задание 3:
Косинусное растояние GN: 0.572

Задание 4:
GN: 0.707
BR: 0.673
Удалено пар: 2

